Notebook for harry testing

In [8]:
import pandas as pd
import numpy as np
import sqlite3

db_path = r"\wsl.localhost\Ubuntu\home\harry\MMSE R&I\laliga\laliga.sqlite"

# Step 1: Connect to the SQLite database



OperationalError: unable to open database file